# Berlin Net trained on the Librivox Dataset
Does this lead to better robustness towards the guru dataset?

**Conclusion:** No

In [2]:
from comet_ml import Experiment
import keras
from keras import models, layers
from keras.optimizers import RMSprop
import numpy as np
from kapre.time_frequency import Melspectrogram
from kapre.augmentation import AdditiveNoise
from kapre.utils import Normalization2D
from keras import regularizers

import sys
import os
sys.path.append('../')
from utils import DataFeed

# to avoid different initizialization of weights
np.random.seed(42)

In [6]:
class AdditionalValidationSets(keras.callbacks.Callback):
    def __init__(self, validation_sets, verbose=0, batch_size=None):
        """
        :param validation_sets:
        a list of 3-tuples (validation_data, validation_targets, validation_set_name)
        or 4-tuples (validation_data, validation_targets, sample_weights, validation_set_name)
        :param verbose:
        verbosity mode, 1 or 0
        :param batch_size:
        batch size to be used when evaluating on the additional datasets
        """
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        for validation_set in self.validation_sets:
            if len(validation_set) not in [2, 3]:
                raise ValueError()
        self.epoch = []
        self.history = {}
        self.verbose = verbose
        self.batch_size = batch_size

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        # record the same values as History() as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            if len(validation_set) == 3:
                validation_data, validation_targets, validation_set_name = validation_set
                sample_weights = None
            elif len(validation_set) == 4:
                validation_data, validation_targets, sample_weights, validation_set_name = validation_set
            else:
                raise ValueError()
    
            results = self.model.evaluate(x=validation_data,
                                          y=validation_targets,
                                          verbose=self.verbose,
                                          batch_size=self.batch_size)

            print(f"{validation_set_name}: {model.metrics_names[0]} = {results[0]},  {model.metrics_names[1]} = {results[1]}")

In [10]:
data_path = '../preprocessing/preprocessed_data'

training_generator = DataFeed.DataGenerator(data_path, ['train/librivox'], batch_size=32)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['test/guru'], num=-1, shuffle=True)
val_data2, val_labels2 = DataFeed.Dataset.create(data_path, ['val/voxforge'], num=-1, shuffle=True)

In [15]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('berlin_net_librivox.h5', monitor='val_loss', save_best_only=True),
             AdditionalValidationSets([(val_data2, val_labels2, 'voxforge')], verbose=0)]

In [16]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=50, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model.add(Normalization2D(str_axis='freq'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))
model.add(layers.Flatten())
model.add(layers.Dense(1048, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=RMSprop(),
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [ ]:
history = model.fit_generator(training_generator,
                              epochs=8,
                              validation_data=(val_data[:-1], val_labels[:-1]), 
                              shuffle=True,
                              callbacks=callbacks)

Epoch 1/8
4567/4567 [==============================] - 213s 47ms/step - loss: 0.5194 - acc: 0.7917 - val_loss: 0.7195 - val_acc: 0.6667
voxforge: loss = 0.8354952196121216,  acc = 0.6268
Epoch 2/8
4567/4567 [==============================] - 211s 46ms/step - loss: 0.3520 - acc: 0.8827 - val_loss: 0.7343 - val_acc: 0.7536
voxforge: loss = 0.8962148693084717,  acc = 0.6172
Epoch 3/8
4567/4567 [==============================] - 213s 47ms/step - loss: 0.3562 - acc: 0.8841 - val_loss: 0.6813 - val_acc: 0.7391
voxforge: loss = 0.7341104091644287,  acc = 0.6844
Epoch 4/8
4567/4567 [==============================] - 209s 46ms/step - loss: 0.3710 - acc: 0.8812 - val_loss: 0.8314 - val_acc: 0.7246
voxforge: loss = 0.7872097562789917,  acc = 0.656
Epoch 5/8
4567/4567 [==============================] - 209s 46ms/step - loss: 0.3969 - acc: 0.8746 - val_loss: 1.3334 - val_acc: 0.6522
voxforge: loss = 0.7914732672691345,  acc = 0.68
Epoch 6/8
 751/4567 [===>..........................] - ETA: 2:44 - l